In [ ]:
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd
import time

In [ ]:
API_KEY = "insert your API Key here"

In [ ]:
url = f"https://www.alphavantage.co/query?function=TOP_GAINERS_LOSERS&apikey={API_KEY}"
response = requests.get(url).json()
print(response)  # Print full response

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


In [ ]:
#function to fetch stock data
def get_stock_data(symbols):
    stock_list = []

    for symbol in symbols:
        url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={API_KEY}"
        response = requests.get(url).json()

        try:
            stock_info = response["Global Quote"]
            stock_list.append({
                "Stock Ticker": stock_info["01. symbol"],
                "Company": symbol,  # Alpha Vantage doesn't provide company names
                "Current Market Price": f"${float(stock_info['05. price']):,.2f}"
            })
        except KeyError:
            print(f"Error: Could not fetch data for {symbol}")

        time.sleep(1)  # Avoid rate limit

    return pd.DataFrame(stock_list)

In [ ]:
#Get user input
symbols = input("Enter at least two stock symbols (comma-separated): ").split(",")

#Fetch and display basic stock data
df_basic = get_stock_data(symbols)
print("\n Basic Stock Data:\n", df_basic)

Enter at least two stock symbols (comma-separated): NVDA,AMZN
Error: Could not fetch data for NVDA
Error: Could not fetch data for AMZN

 Basic Stock Data:
 Empty DataFrame
Columns: []
Index: []


In [ ]:
#function to fetch additional stock details
def get_stock_details(symbol, module):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_KEY}"
    response = requests.get(url).json()

    try:
        if module == "52 Week High":
            return response["52WeekHigh"]
        elif module == "52 Week Low":
            return response["52WeekLow"]
        elif module == "Return on Assets (ROA)":
            return response["ReturnOnAssetsTTM"]
    except KeyError:
        return "N/A"

#Fetch additional details for a chosen stock
chosen_stock = input("\nEnter a stock symbol to fetch additional details: ").upper()
chosen_module = input("Choose a module (52 Week High, 52 Week Low, Return on Assets (ROA)): ")
detail = get_stock_details(chosen_stock, chosen_module)
print(f"\n {chosen_module} for {chosen_stock}: {detail}")

# Create and display a DataFrame with the result
df_result = pd.DataFrame({
    'Stock Ticker': [chosen_stock],
    'Module': [chosen_module],
    'Value': [detail]
})

print('\nAdditional Stock Data:')
print(df_result)


Enter a stock symbol to fetch additional details: GOOGL
Choose a module (52 Week High, 52 Week Low, Return on Assets (ROA)): 52 Week High

 52 Week High for GOOGL: 207.05

Additional Stock Data:
  Stock Ticker        Module   Value
0        GOOGL  52 Week High  207.05


In [ ]:
def get_stock_data_for_symbol(symbol):
    # Call GLOBAL_QUOTE endpoint to get current price
    url_quote = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={API_KEY}'
    response_quote = requests.get(url_quote).json()
    try:
        quote = response_quote['Global Quote']
        current_price = float(quote['05. price'])
    except KeyError:
        current_price = None

    # Call OVERVIEW endpoint to get additional details (company name, 52-week high/low)
    url_overview = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_KEY}'
    response_overview = requests.get(url_overview).json()
    try:
        company = response_overview['Name']
        week_high = response_overview['52WeekHigh']
        week_low = response_overview['52WeekLow']
    except KeyError:
        company = symbol  # fallback
        week_high = None
        week_low = None

    return {
        'Stock Ticker': symbol,
        'Company': company,
        'Current Market Price': current_price,
        '52 Week High': week_high,
        '52 Week Low': week_low
    }

# I can't figure our how to get the API that I chose to really try for tredning stocks in the whole list of possibilities...
# I have also hit the free rate limit of alpha vantage unfortunately so I can no longer see if my code is working... I think this is why I am returning errors
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'NFLX', 'BRK.B', 'JNJ', 'F', 'WBD', 'AAL','T','RKLB','RDDT']

stock_data = []
for symbol in symbols:
    data = get_stock_data_for_symbol(symbol)
    if data['Current Market Price'] is not None:
        stock_data.append(data)
    else:
        print(f'Error fetching data for {symbol}')
    time.sleep(12)  # Delay to respect API rate limits

# Convert the results to a DataFrame
df_stocks = pd.DataFrame(stock_data)

# Sort the DataFrame by current price (highest first) and pick the top 5
df_trending = df_stocks.sort_values(by='Current Market Price', ascending=False).head(5)

# Format the current price as currency for display
df_trending['Current Market Price'] = df_trending['Current Market Price'].apply(lambda x: f'${x:,.2f}')

print('\n Trending Stocks (Top 5 by Current Price):\n')
print(df_trending[['Stock Ticker', 'Company', 'Current Market Price', '52 Week High', '52 Week Low']])

Error fetching data for TSLA
Error fetching data for NVDA
Error fetching data for META
Error fetching data for NFLX
Error fetching data for BRK.B
Error fetching data for JNJ
Error fetching data for F
Error fetching data for WBD
Error fetching data for AAL
Error fetching data for T
Error fetching data for RKLB
Error fetching data for RDDT

 Trending Stocks (Top 5 by Current Price):

  Stock Ticker                Company Current Market Price 52 Week High  \
1         MSFT  Microsoft Corporation              $399.73       465.64   
0         AAPL              Apple Inc              $240.36       259.81   
3         AMZN                   AMZN              $214.35         None   
2        GOOGL   Alphabet Inc Class A              $172.73       207.05   

  52 Week Low  
1      383.35  
0      163.31  
3        None  
2      130.19  
